# client

> SDK to interact with the Agora-Fewsats Marketplace API

In [ ]:
#| default_exp client

In [ ]:
import os
import httpx
import json
import uuid
from typing import Dict, List, Optional, Any, Union
from agora.config import *
from agora.crypto import *

from fastcore.utils import *


In [ ]:
# Test values

test_product_id = "678f61ec0205c9203bc07a45"
test_variant_id = 42884194664644

test_shipping_address = {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            }
test_user = {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }

In [ ]:
#| export


class Agora:
    """
    Client for the Agora-Fewsats Marketplace API.
    
    This client provides methods to interact with the Agora-Fewsats Marketplace API,
    including product search, cart management, and checkout functionality.
    """
    
    def __init__(self, base_url: str = "https://agora-backend.replit.app/api/v1", private_key: str = ''):
        """
        Initialize the Agora Marketplace client.
        
        Args:
            base_url: The base URL of the Agora-Fewsats Backend API.
            private_key: The private key of the Agora-Fewsats Backend API. If not provided,
                         a random UUID will be generated.
        """
        self.base_url = base_url

        cfg = get_cfg()

        if private_key: self.pk, self.pub = from_pk_hex(private_key) # if provided use the private key
        elif cfg.priv: self.pk, self.pub = from_pk_hex(cfg.priv) # if not provided use the private key from the config file
        else: 
            self.pk, self.pub = generate_keys()
            save_cfg({'priv': priv_key_hex(self.pk)})

        self.mongo_id = self.pub[:24] # agora uses mongoDB so the related user-objectId needs to be 24 characters

        self.client = httpx.Client(timeout=30.0)
        self.client.headers.update({"customuserid": self.mongo_id})

    def _make_request(self, method: str, endpoint: str, **kwargs) -> httpx.Response:
        """
        Make a request to the Agora-Fewsats Backend API.
        
        Args:
            method: The HTTP method to use.
            endpoint: The API endpoint to call.
            **kwargs: Additional arguments to pass to httpx.
            
        Returns:
            The response from the API.
        """
        url = f"{self.base_url}/{endpoint}"
        response = self.client.request(method, url, **kwargs)
        
        
        return response
        
    def search_products(self, query: str, count: int = 20, page: int = 1, 
                         price_min: int = 0, price_max: Optional[int] = None, 
                         sort: Optional[str] = None, order: Optional[str] = None) -> Dict:
        """
        Search for products.
        
        Args:
            query: The search query.
            count: The number of products to return per page.
            page: The page number.
            price_min: The minimum price.
            price_max: The maximum price.
            sort: The sort field.
            order: The sort order.
            
        Returns:
            The search results.
        """
        params = {
            "q": query,
            "count": count,
            "page": page,
            "price_min": price_min
        }
        
        if price_max is not None:
            params["price_max"] = price_max
        if sort is not None:
            params["sort"] = sort
        if order is not None:
            params["order"] = order
            
        return self._make_request("GET", "search", params=params)


In [ ]:
a = Agora()
# a = Agora(base_url="http://localhost:8000/api/v1")
r = a.search_products("shirt")
p = r.json()["Products"][0]
p

{'_id': '678cfe72202868c9fd812284',
 'name': 'Favorite Shirt',
 'storeName': 'Harper & Tucker',
 'brand': 'Stillwater',
 'slug': 'favorite-shirt-a97905a6-11c3-4fb8-8fec-86adf3b97517-1737293426144',
 'price': 150,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/1403/9469/files/FavoriteShirt-WhitePoplin-Front.png?v=1722961682'],
 'url': 'https://harperandtucker.com/products/favorite-shirt',
 'agoraScore': 92,
 'priceHistory': [{'price': 150,
   'date': '2025-01-19T13:30:26.169Z',
   '_id': '678cfe72202868c9fd81246c'}],
 '_rankingScore': 0.72265625,
 '_combinedScoreData': {'clipScore': 0.677734375,
  'cohereScore': 0.72265625,
  'maxScore': 0.72265625,
  'bestEmbedding': 'cohere',
  'position': 0},
 '_adjustedScore': 0.72265625}

In [ ]:
#| export

@patch    
def get_product_detail(self: Agora, slug: str) -> Dict:
    """
    Get details for a specific product.
    
    Args:
        slug: The product slug.
        
    Returns:
        The product details.
    """
    params = {"slug": slug}
    return self._make_request("GET", "product-detail", params=params)


In [ ]:
r = a.get_product_detail(p['slug'])
r.json()

{'status': 'success',
 'product': {'_id': '678cfe72202868c9fd812284',
  'keywords': [],
  'name': 'Favorite Shirt',
  'storeName': 'Harper & Tucker',
  'brand': 'Stillwater',
  'tags': ['Core', 'Foundations', 'Spring', 'Summer'],
  'description': '\n\n<div>\n<div>\n STYLE\n<div>Button down shirt <br />Collared <br />Two breast pockets <br />Long hemline can be tucked in or worn loose <br />Long sleeves <br />Unlined</div>\n</div>\n<div>\n FIT\n<div>Relaxed, oversized fit</div>\n</div>\n<div>\n MATERIALS &amp; PRODUCTION\n<div>Gauze Fabrication - 100% Cotton\xa0<br /><br />Made in Los Angeles, CA</div>\n</div>\n</div>',
  'country': 'US',
  'currency': 'USD',
  'slug': 'favorite-shirt-a97905a6-11c3-4fb8-8fec-86adf3b97517-1737293426144',
  'price': '150.00',
  'externalProductId': '8239479095526',
  'published_at': '2024-04-24T13:38:32.000Z',
  'last_updated': '2025-01-19T13:30:25.000Z',
  'verifiedAt': '2025-01-19T13:30:26.144Z',
  'isVerified': False,
  'isBoosted': False,
  'source': 

In [ ]:
#| export

@patch
def get_cart(self: Agora) -> Dict:
    """
    Get the current user's cart.
    
    Returns:
        The cart details.
    """
    return self._make_request("GET", "cart")


In [ ]:
r = a.get_cart()
r.json()

{'id': 1,
 'customuserid': '1dd605e0a67f966bab10ceea',
 'created_at': 1742573592,
 'updated_at': 1742573592,
 'items': []}

In [ ]:
#| export
  
@patch
def add_to_cart(self: Agora, slug: str, product_id: str, variant_id: str = None, quantity: int = 1) -> Dict:
    """
    Add an item to the user's cart. Somem products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to add.
        
    Returns:
        The updated cart.
    """
    item = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id if variant_id else product_id,
        "quantity": quantity,
    }
    
    return self._make_request("POST", "cart/items", json=item)


In [ ]:
r = a.add_to_cart(p['slug'], p['_id'])
r.status_code, r.json()

(200,
 {'id': 1,
  'customuserid': '1dd605e0a67f966bab10ceea',
  'created_at': 1742573592,
  'updated_at': 1742573592,
  'items': [{'id': 1,
    'product_id': '678cfe72202868c9fd812284',
    'variant_id': '678cfe72202868c9fd812284',
    'quantity': 1,
    'title': 'Favorite Shirt by Stillwater',
    'description': 'Favorite Shirt by Stillwater from Harper & Tucker',
    'amount': 15000,
    'currency': 'USD',
    'slug': 'favorite-shirt-a97905a6-11c3-4fb8-8fec-86adf3b97517-1737293426144'}]})

In [ ]:
#| export

@patch
def update_cart_item(self: Agora, slug: str, product_id: str, variant_id: str, quantity: int) -> Dict:
    """
    Update the quantity of an item in the cart. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The new quantity.
        
    Returns:
        The updated cart.
    """
    params = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity
    }
    
    return self._make_request("PUT", "cart/items", params=params)
    


In [ ]:
r = a.update_cart_item(p['slug'], p['_id'], p['_id'], 1)
r.status_code, r.json()


(200,
 {'id': 1,
  'customuserid': '1dd605e0a67f966bab10ceea',
  'created_at': 1742573592,
  'updated_at': 1742573593,
  'items': [{'id': 1,
    'product_id': '678cfe72202868c9fd812284',
    'variant_id': '678cfe72202868c9fd812284',
    'quantity': 1,
    'title': 'Favorite Shirt by Stillwater',
    'description': 'Favorite Shirt by Stillwater from Harper & Tucker',
    'amount': 15000,
    'currency': 'USD',
    'slug': 'favorite-shirt-a97905a6-11c3-4fb8-8fec-86adf3b97517-1737293426144'}]})

In [ ]:
#| export

@patch
def clear_cart(self: Agora) -> Dict:
    """
    Clear all items from the cart.
    
    Returns:
        The response from the API.
    """
    return self._make_request("DELETE", "cart")

In [ ]:
r = a.clear_cart()
r.status_code, r.json()

(200, {'status': 'success', 'data': None, 'message': 'Cart cleared'})

In [ ]:
#| export

@patch
def buy_now(self: Agora, slug: str, product_id: str, variant_id: str, shipping_address: Dict,
            user: Dict, quantity: int = 1) -> Dict:
    """
    Purchase a product directly. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to purchase.
        shipping_address: The shipping address.
        user: The user information.
        
    Example:
        shipping_address = {
            "addressFirst": "123 Main St",
            "city": "New York",
            "state": "NY",
            "country": "US",
            "addressName": "Home",
            "zipCode": "10001"
        }
        
        user = {
            "firstname": "John",
            "lastname": "Doe",
            "email": "john@example.com",
            "_id": "user123"
        }
        
        a.buy_now(product_id, variant_id, shipping_address, user)
        
    Returns:
        The payment information.
    """

        
    request_data = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity,
        "shipping_address": shipping_address,
        "user": user
    }
    
    return self._make_request("POST", "buy-now", json=request_data)

In [ ]:
r = a.buy_now(p['slug'], p['_id'], p['_id'], test_shipping_address, test_user, quantity=1)
r.status_code, r.json()

(402,
 {'offers': [{'id': 'c5d7971e-a958-46cd-971b-e1f477bbc0c3',
    'amount': 15000,
    'currency': 'USD',
    'description': 'Favorite Shirt by Stillwater from Harper & Tucker',
    'title': 'Favorite Shirt by Stillwater',
    'payment_methods': ['lightning', 'credit_card'],
    'type': 'one-off'}],
  'payment_context_token': 'cd286fc4-fdd5-4dff-a806-d0e61b6b8fb5',
  'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
  'version': '0.2.2'})

In [ ]:
# test webhook 
offers = r.json()

from datetime import datetime, timezone

# Your existing data from buy_now
offer_id = offers['offers'][0]['id']
payment_context_token = offers['payment_context_token']
amount = offers['offers'][0]['amount']
currency = offers['offers'][0]['currency']

# Create a webhook payload
payload = {
    "offer_id": offer_id,
    "payment_context_token": payment_context_token,
    "amount": amount,
    "currency": currency,
    "status": "succeeded",  # Use 'succeeded' to simulate successful payment
    "timestamp": datetime.now(timezone.utc).isoformat()
}

# Send the request to your webhook endpoint
url = f"{a.base_url}/webhook/payment"  # Adjust if your server is running elsewhere
# url = "http://localhost:8000/api/v1/webhook/payment"  # Adjust if your server is running elsewhere
response = httpx.post(url, json=payload)


In [ ]:
#| export
   
@patch
def get_order(self: Agora, order_id: int) -> Dict:
    """
    Get details for a specific order.
    
    Args:
        order_id: The order ID.
        
    Returns:
        The order details.
    """
    return self._make_request("GET", f"orders/{order_id}")
    

@patch
def get_user_orders(self: Agora) -> List[Dict]:
    """
    Get all orders for the current user.
    
    Returns:
        A list of orders.
    """
    return self._make_request("GET", f"users/{self.customer_id}/orders")
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()